In [ ]:
# this cell reads the chunks files where data is organised in columns as follows
# L, time, realisations in this chunk, moment1, moment2, moment3, moment4, moment5, moment6
# the name of the files contains *time_moments*
# this cell computes the mean and standard errorof the mean of each moment and writes the result in a file
# with probability 1 this piece of code can be made more efficient

import pandas as pd #this is for tabular data munging
import numpy as np#
import glob # file listings
#import matplotlib.pyplot as plt

#%matplotlib inline

def filterdf(df):
    return df.tail(2)

systemsize = 63
L=str(systemsize)
result = []
for i in range(5,11): # iterate through the different time series we are interested in
                      # (in this case, every column corresponds to a different time series)
    alldfs = []
    s = 0
    for f in glob.glob('./../L'+L+'/*time_moments*'): #iterate through all the files containing *time_moments*
        df = pd.read_csv(f, sep="\t", comment="#", header=None) #read the file
        dfi = df.ix[:,i] #extract the time series in column i
        alldfs.append(dfi) #append time series to alldfs
        if (len(dfi)>s):
            t = df.ix[:,2] # read the time column in the time series and keep the longest one (all time series share the
                           # same temporal separation but the final time of the realisations is different in each series)
            s = len(dfi)

    ndf = pd.concat(alldfs,axis=1)
    [rows,cols] = ndf.shape
    ndf.columns = list(range(cols))

    l = min(pd.isnull(ndf).any(1).nonzero()[0])
    for j in range(0,cols): #complete those time series that finished before the others by copying the las value
        for k in range(l,rows):
            if pd.isnull(ndf.ix[k,j]):
                ndf.ix[k,j] = ndf.ix[k-1,j]

    N = cols
    dfmean = np.mean(ndf,axis=1) #compute the columns of mean values
    dfstd = np.std(ndf,axis=1) #compute the column of variances
    error = dfstd/np.sqrt(N) #compute the column of errors
    if (i == 5): #copy the time column only once, in the first loop
        result.append(t)
    result.append(dfmean) #copy the column of means
    result.append(error) #copy the column of errors
    
df = pd.DataFrame(result).transpose()
df['L'] = systemsize
df.columns=['time','1mom','1mom_err','2mom','2mom_err','3mom','3mom_err','4mom','4mom_err','5mom','5mom_err','6mom','6mom_err', '#L']
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]  #    OR    df = df.ix[:, cols]
#df.to_csv('./codev6_20161118/results/elaborated_results/L32767_radius_gyration_moments_errors.dat',sep='\t',index=False)
df.to_csv('./L'+L+'_visited_sites_moments_errors.dat',sep='\t',index=False) #save in file
#df